# How to use OpenCV to segment the cells

## Compiler parameters

Set my Jypyter environment for the use of OpenCV in a C++ notebook. You don't need this line when you write yur own C++ programs. I need it to set my interactive compiler ([Cling](https://root.cern/cling/)). For your own program, use CMake.

In [1]:
#include "../../../includeLibraries.h"

## Header inclusion for C++

In [2]:
#include <iostream>

#include <opencv2/opencv.hpp>
#include <opencv2/imgproc.hpp>

## Add the namespaces

In [3]:
using namespace std;

In [4]:
using namespace cv;

## Read the image from a file on the disk and return a new matrix

![Input image.](../cells_greyscale.png)

In [5]:
Mat image = imread("../cells_greyscale.png", IMREAD_GRAYSCALE);

## Check for errors

In [6]:
// Check for failure
if (image.empty()) 
{
    cerr << "Could not open or find the image" << endl;
    return -1;
}

## Compute its histogram

In [7]:
Mat histogram;

int histogram_size = 255;

calcHist(&image, // Pointer on the image
         1, // Number of images
         0, // Number of channels
         Mat(), // Binary mask
         histogram, // Output
         1, // Number of dimensions
         &histogram_size, // Number of bins
         0); // Ranges

## Display the image in a window

In [8]:
namedWindow("Cells", WINDOW_GUI_EXPANDED); // Create a window
imshow("Cells", image); // Show our image inside the created window.

## Plot the histogram

In [9]:
Mat histogram_image = Mat::ones(200, 320, CV_8U) * 255;
normalize(histogram, histogram, 0, histogram_image.rows, NORM_MINMAX, CV_32F);
histogram_image = Scalar::all(255);
int binW = cvRound(double(histogram_image.cols) / histogram_size);
for( int i = 0; i < histogram_size; i++ )
    rectangle(histogram_image,
              Point(i*binW, histogram_image.rows),
              Point((i+1)*binW, histogram_image.rows - cvRound(histogram.at<float>(i))),
              Scalar::all(0), -1, 8, 0 );

imshow("histogram", histogram_image);

In [10]:
waitKey(0); // Wait for any keystroke in the window
destroyWindow("Cells"); // Destroy the created window
destroyWindow("histogram"); // Destroy the created window

A window will open:

![OpenCV window](../opencv-cxx-load-display.png)

## Binary threshold

A lot of information about the various thresholds technique in OpenCV is available at [https://docs.opencv.org/master/db/d8e/tutorial_threshold.html](https://docs.opencv.org/master/db/d8e/tutorial_threshold.html) on OpenCV's website. We want the background in black, and the cells in white. We use the inverse of the binary threshold then. See `THRESH_BINARY_INV` in [https://docs.opencv.org/master/d7/d1b/group__imgproc__misc.html#gaa9e58d2860d4afa658ef70a9b1115576](https://docs.opencv.org/master/d7/d1b/group__imgproc__misc.html#gaa9e58d2860d4afa658ef70a9b1115576).

In [11]:
int threshold_value = 70; // Taken from the histogram
int threshold_type = THRESH_BINARY_INV;
int const max_value = 255;
int const max_type = 4;
Mat segmentation;

threshold(image,
          segmentation,
          threshold_value,
          max_value,
          threshold_type);

## Display the result

In [12]:
namedWindow("Cells", WINDOW_GUI_EXPANDED); // Create a window
imshow("Cells", image); // Show our image inside the created window.

namedWindow("Segmentation", WINDOW_GUI_EXPANDED); // Create a window
imshow("Segmentation", segmentation); // Show our image inside the created window.

waitKey(0); // Wait for any keystroke in the window
destroyAllWindows(); // Destroy all the created windows

You'll see:

![opencv-cxx-threshold1.png](../opencv-cxx-threshold1.png)

which is not perfect. Remember mathematical morphology? Opening? Closing? Have a look at [https://docs.opencv.org/master/d3/dbe/tutorial_opening_closing_hats.html](https://docs.opencv.org/master/d3/dbe/tutorial_opening_closing_hats.html). 

And let's do some cleaning.

## Cleaning

In [13]:
Mat element = getStructuringElement(MORPH_CROSS,
                                    Size(5, 5), 
                                    Point(2, 2));

morphologyEx(segmentation, segmentation, MORPH_CLOSE, element);
morphologyEx(segmentation, segmentation, MORPH_OPEN, element);

In [14]:
namedWindow("Cells", WINDOW_GUI_EXPANDED); // Create a window
imshow("Cells", image); // Show our image inside the created window.

namedWindow("Segmentation", WINDOW_GUI_EXPANDED); // Create a window
imshow("Segmentation", segmentation * 255); // Show our image inside the created window.

waitKey(0); // Wait for any keystroke in the window
destroyAllWindows(); // Destroy all the created windows

You'll see:

![opencv-cxx-threshold2.png](../opencv-cxx-threshold2.png)

which is much better than the previous one.

## Apply the mask

The threshold provide a binary mask. 

In [15]:
Mat image_of_cells;
bitwise_and(segmentation, image, image_of_cells);

In [16]:
namedWindow("Cells", WINDOW_GUI_EXPANDED); // Create a window
imshow("Cells", image); // Show our image inside the created window.

namedWindow("Segmentation", WINDOW_GUI_EXPANDED); // Create a window
imshow("Segmentation", segmentation); // Show our image inside the created window.

namedWindow("Just the cells", WINDOW_GUI_EXPANDED); // Create a window
imshow("Just the cells", image_of_cells); // Show our image inside the created window.

waitKey(0); // Wait for any keystroke in the window
destroyAllWindows(); // Destroy all the created windows

You'll see:

![opencv-cxx-threshold3.png](../opencv-cxx-threshold3.png)

The background has been removed.